In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.layers import SimpleRNN
from keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Подготовка датасета


In [ ]:
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/medium_articles.csv")
print(data)

                                            title  \
0                             Mental Note Vol. 24   
1                       Your Brain On Coronavirus   
2                                  Mind Your Nose   
3                        The 4 Purposes of Dreams   
4                Surviving a Rod Through the Head   
...                                           ...   
192363        Why do you need a cleaning service?   
192364  Daily cleaning and maintenance of bedding   
192365        Beneficial Advice on Bond Cleaning!   
192366    How I Learned Romanian in 37 Easy Steps   
192367     Trying Pimsleur Cantonese in Hong Kong   

                                                     text  \
0       Photo by Josh Riemer on Unsplash\n\nMerry Chri...   
1       Your Brain On Coronavirus\n\nA guide to the cu...   
2       Mind Your Nose\n\nHow smell training can chang...   
3       Passionate about the synergy between science a...   
4       You’ve heard of him, haven’t you? Phineas Gage... 

In [ ]:
from collections import Counter
def pars_tags (data):
  d = {}
  for el in data['tags']:
    for i in el:
      if i in d:
        d[i] += 1
      else:
        d[i] = 1
  cnt = Counter(d).most_common(10)
  return cnt

In [ ]:
def upd_data (data, tag_list):
  new_data = pd.DataFrame()
  for _, el in data.iterrows():
    for i in el['tags']:
      for j in tag_list:
        if i == j[0]:
          el['tags'] = i
          new_data = new_data.append(el, ignore_index = True)
  return new_data     

In [ ]:
tmp = data.tags.apply(lambda x: x[1:-1].split(','))
tmp_data = data.copy() 
tmp_data['tags'] = tmp
tags_list = pars_tags(tmp_data)
new_data = upd_data(tmp_data, tags_list)
print(new_data)

<ipython-input-69-c693402706d4>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_data = new_data.append(el, ignore_index = True)


                                                   title  \
0      How to Turn Your Popular Blog Series Into a Be...   
1              Four Exercises to Strengthen Your Writing   
2      This 10-Minute Routine Will Increase Your Clar...   
3              How to Make Your Day Job Support Your Art   
4      An Effective Five-Step Process for Writing Cap...   
...                                                  ...   
50163               bStable needs you 🦸‍♂️ 🧑‍✈️🧑‍🔬🧑‍💻👨‍🌾   
50164               bStable needs you 🦸‍♂️ 🧑‍✈️🧑‍🔬🧑‍💻👨‍🌾   
50165                Status messenger for Desktop review   
50166  The Impact of Technology on Hotel Guest Experi...   
50167  (Live — Streaming) : 2021 Soul Train Music Awards   

                                                    text  \
0      How to Turn Your Popular Blog Series Into a Be...   
1      A few months ago, I wrote an article sharing s...   
2      “Your subconscious mind works continuously, wh...   
3      Gather around kids and let me te

In [ ]:
x = new_data['text']
y = new_data['tags']
encoder = LabelEncoder()
y = encoder.fit_transform(y)
new_data = pd.DataFrame(x)
new_data['tags'] = y
new_data = new_data[:200]
print(new_data)
train, test = train_test_split(new_data, test_size=0.33, random_state=42)

                                                  text  tags
0    How to Turn Your Popular Blog Series Into a Be...     9
1    A few months ago, I wrote an article sharing s...     9
2    “Your subconscious mind works continuously, wh...     4
3    Gather around kids and let me tell you a story...     9
4    You’ve spent hours pouring your heart and soul...     9
..                                                 ...   ...
195  Redefine Publishing\n\nStaying consistent duri...     9
196  The difference between beneficial and destruct...     7
197  Who are Medium’s Curators and What do They do?...     9
198  Section 1: List of Challenges and Related Stor...     9
199  Practical Examples\n\nIterators\n\nIn Python, ...     3

[200 rows x 2 columns]


In [ ]:
x = train["text"]
y = train["tags"]
x = np.asarray(x).astype(np.str_)
y = np.asarray(y).astype(np.float32)
test_x = test["text"]
test_y = test["tags"]
test_x = np.asarray(test_x).astype(np.str_)
test_y = np.asarray(test_y).astype(np.float32)

## Посимвольная векторизация

In [ ]:
vectorizer = tf.keras.layers.TextVectorization(split='character')

print('Training vectorizer')
vectorizer.adapt(list(train["text"]))

Training vectorizer


## Нейросетевая архитектура SimpleRNN

In [ ]:
model = keras.Sequential()
model.add(vectorizer)
model.add(layers.Embedding(input_dim=100000, output_dim=8))
model.add(layers.SimpleRNN(4))
model.add(layers.Dense(100, activation = 'relu'))
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_3 (TextV  (None, None)             0         
 ectorization)                                                   
                                                                 
 embedding_3 (Embedding)     (None, None, 8)           800000    
                                                                 
 simple_rnn_3 (SimpleRNN)    (None, 4)                 52        
                                                                 
 dense_3 (Dense)             (None, 100)               500       
                                                                 
Total params: 800,552
Trainable params: 800,552
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(keras.optimizers.Adam(0.01), keras.losses.SparseCategoricalCrossentropy(from_logits=True), ['acc'])
train_model = model.fit(x=x, y=y, batch_size=32, epochs = 7, validation_data=(test_x, test_y))

Epoch 1/7
5/5 [==============================] - 166s 33s/step - loss: 4.6040 - acc: 0.0000e+00 - val_loss: 4.5522 - val_acc: 0.0455
Epoch 2/7
5/5 [==============================] - 159s 32s/step - loss: 4.5004 - acc: 0.0448 - val_loss: 4.4256 - val_acc: 0.5758
Epoch 3/7
5/5 [==============================] - 135s 24s/step - loss: 4.3510 - acc: 0.6269 - val_loss: 4.2707 - val_acc: 0.5909
Epoch 4/7
5/5 [==============================] - 150s 28s/step - loss: 4.1769 - acc: 0.6418 - val_loss: 4.1048 - val_acc: 0.6061
Epoch 5/7
5/5 [==============================] - 152s 28s/step - loss: 3.9839 - acc: 0.6418 - val_loss: 3.9340 - val_acc: 0.6061
Epoch 6/7
5/5 [==============================] - 152s 30s/step - loss: 3.7989 - acc: 0.6418 - val_loss: 3.7585 - val_acc: 0.6061
Epoch 7/7
5/5 [==============================] - 148s 31s/step - loss: 3.5950 - acc: 0.6418 - val_loss: 3.5802 - val_acc: 0.6061


## Пословная векторизация

In [ ]:
vectorizer = tf.keras.layers.TextVectorization(split='whitespace')

print('Training vectorizer')
vectorizer.adapt(list(train["text"]))

Training vectorizer


## Simple

In [ ]:
model = keras.Sequential()
model.add(vectorizer)
model.add(layers.Embedding(input_dim=100000, output_dim=4))
model.add(layers.SimpleRNN(8))
model.add(layers.Dense(50, activation = 'relu'))
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_4 (TextV  (None, None)             0         
 ectorization)                                                   
                                                                 
 embedding_4 (Embedding)     (None, None, 4)           400000    
                                                                 
 simple_rnn_4 (SimpleRNN)    (None, 8)                 104       
                                                                 
 dense_4 (Dense)             (None, 50)                450       
                                                                 
Total params: 400,554
Trainable params: 400,554
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(keras.optimizers.Adam(0.01), keras.losses.SparseCategoricalCrossentropy(from_logits=True), ['acc'])
train_model = model.fit(x=x, y=y, batch_size=32, epochs = 4, validation_data=(test_x, test_y))

Epoch 1/4
5/5 [==============================] - 19s 4s/step - loss: 3.8900 - acc: 0.0299 - val_loss: 3.8376 - val_acc: 0.1364
Epoch 2/4
5/5 [==============================] - 17s 3s/step - loss: 3.7785 - acc: 0.4104 - val_loss: 3.7014 - val_acc: 0.5909
Epoch 3/4
5/5 [==============================] - 17s 3s/step - loss: 3.6470 - acc: 0.6418 - val_loss: 3.5502 - val_acc: 0.6061
Epoch 4/4
5/5 [==============================] - 17s 3s/step - loss: 3.4546 - acc: 0.6493 - val_loss: 3.3186 - val_acc: 0.6061
